# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** The CNN architecture for the encoder is the pretrained RESNET-50 model. The RNN structure for the decoder is a single layer of LSTM as per the paper "Show and Tell". I chose vocab_threshold=4 so as to have a larger dictionary. Other hyperparameters like hidden_size = 512 and embed_size = 512 are standard numbers used in machine learning literature. I chose a large batch_size = 256, so as to achieve faster training. 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I used the existing transform in `transform_train` as it looked okay to train the model.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** The trainable parameters list(encoder.embed.parameters()) is used for training as we are using a pretrained model which was trained on Imagenet and training only the last layer of CNN to get the embeddings. list(decoder.parameters()) is used to train the decoder as it's not a pretrained model ans we need to train it from scratch.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** Adam is a standard optimizer which has a decent performance for most applications.

In [2]:
import nltk
nltk.download('punkt')
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
## TODO #1: Select appropriate values for the Python variables below.
batch_size = 256          # batch size
vocab_threshold = 4        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512         # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

In [7]:
# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer =  torch.optim.Adam(params, lr=.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 385/414113 [00:00<01:47, 3846.25it/s]

  0%|          | 791/414113 [00:00<01:45, 3906.80it/s]

Done (t=0.90s)
creating index...
index created!
Obtaining caption lengths...




  0%|          | 1211/414113 [00:00<01:43, 3988.21it/s]

  0%|          | 1620/414113 [00:00<01:42, 4017.25it/s]

  0%|          | 2037/414113 [00:00<01:41, 4061.09it/s]

  1%|          | 2430/414113 [00:00<01:44, 3941.04it/s]

  1%|          | 2866/414113 [00:00<01:41, 4057.98it/s]

  1%|          | 3287/414113 [00:00<01:40, 4100.78it/s]

  1%|          | 3705/414113 [00:00<01:39, 4123.11it/s]

  1%|          | 4127/414113 [00:01<01:38, 4148.50it/s]

  1%|          | 4557/414113 [00:01<01:37, 4190.70it/s]

  1%|          | 4990/414113 [00:01<01:36, 4231.15it/s]

  1%|▏         | 5409/414113 [00:01<01:36, 4216.78it/s]

  1%|▏         | 5831/414113 [00:01<01:36, 4216.48it/s]

  2%|▏         | 6251/414113 [00:01<01:38, 4140.17it/s]

  2%|▏         | 6664/414113 [00:01<01:39, 4108.99it/s]

  2%|▏         | 7075/414113 [00:01<01:39, 4078.81it/s]

  2%|▏         | 7483/414113 [00:01<01:40, 4044.75it/s]

  2%|▏         | 7888/414113 [00:01<01:40, 4035.66it/s]

  2%|▏         | 8299/414113 

 13%|█▎        | 55646/414113 [00:13<01:22, 4331.56it/s]

 14%|█▎        | 56080/414113 [00:13<01:22, 4324.36it/s]

 14%|█▎        | 56513/414113 [00:13<01:22, 4318.82it/s]

 14%|█▍        | 56946/414113 [00:13<01:23, 4279.93it/s]

 14%|█▍        | 57398/414113 [00:14<01:22, 4347.55it/s]

 14%|█▍        | 57834/414113 [00:14<01:22, 4344.17it/s]

 14%|█▍        | 58269/414113 [00:14<01:22, 4297.76it/s]

 14%|█▍        | 58700/414113 [00:14<01:24, 4219.40it/s]

 14%|█▍        | 59123/414113 [00:14<01:24, 4206.81it/s]

 14%|█▍        | 59550/414113 [00:14<01:23, 4224.84it/s]

 14%|█▍        | 59973/414113 [00:14<01:24, 4210.25it/s]

 15%|█▍        | 60403/414113 [00:14<01:23, 4235.20it/s]

 15%|█▍        | 60827/414113 [00:14<01:23, 4228.08it/s]

 15%|█▍        | 61255/414113 [00:14<01:23, 4243.09it/s]

 15%|█▍        | 61680/414113 [00:15<01:23, 4206.75it/s]

 15%|█▍        | 62106/414113 [00:15<01:23, 4220.09it/s]

 15%|█▌        | 62529/414113 [00:15<01:23, 4202.40it/s]

 15%|█▌       

...

 40%|███▉      | 164642/414113 [00:39<00:58, 4230.60it/s]

 40%|███▉      | 165066/414113 [00:39<00:59, 4217.08it/s]

 40%|███▉      | 165501/414113 [00:39<00:58, 4255.02it/s]

 40%|████      | 165927/414113 [00:39<00:58, 4224.56it/s]

 40%|████      | 166350/414113 [00:39<00:59, 4187.90it/s]

 40%|████      | 166770/414113 [00:39<00:59, 4145.95it/s]

 40%|████      | 167196/414113 [00:39<00:59, 4179.04it/s]

 40%|████      | 167615/414113 [00:40<00:59, 4163.32it/s]

 41%|████      | 168032/414113 [00:40<00:59, 4162.40it/s]

 41%|████      | 168449/414113 [00:40<00:59, 4140.52it/s]

 41%|████      | 168864/414113 [00:40<00:59, 4130.15it/s]

 41%|████      | 169299/414113 [00:40<00:58, 4193.62it/s]

 41%|████      | 169731/414113 [00:40<00:57, 4230.09it/s]

 41%|████      | 170165/414113 [00:40<00:57, 4260.53it/s]

 41%|████      | 170592/414113 [00:41<01:58, 2055.78it/s]

 41%|████▏     | 171026/414113 [00:41<01:39, 2441.02it/s]

 41%|████▏     | 171436/414113 [00:41<01:27, 2778.23it/s

 53%|█████▎    | 218892/414113 [00:52<00:43, 4437.48it/s]

 53%|█████▎    | 219341/414113 [00:52<00:43, 4451.29it/s]

 53%|█████▎    | 219798/414113 [00:52<00:43, 4485.59it/s]

 53%|█████▎    | 220247/414113 [00:52<00:43, 4468.61it/s]

 53%|█████▎    | 220697/414113 [00:52<00:43, 4477.27it/s]

 53%|█████▎    | 221145/414113 [00:52<00:43, 4410.86it/s]

 54%|█████▎    | 221587/414113 [00:53<00:44, 4360.15it/s]

 54%|█████▎    | 222029/414113 [00:53<00:43, 4377.07it/s]

 54%|█████▎    | 222467/414113 [00:53<00:47, 4051.54it/s]

 54%|█████▍    | 222896/414113 [00:53<00:46, 4118.74it/s]

 54%|█████▍    | 223339/414113 [00:53<00:45, 4205.94it/s]

 54%|█████▍    | 223788/414113 [00:53<00:44, 4286.57it/s]

 54%|█████▍    | 224234/414113 [00:53<00:43, 4336.79it/s]

 54%|█████▍    | 224686/414113 [00:53<00:43, 4388.95it/s]

 54%|█████▍    | 225127/414113 [00:53<00:43, 4377.12it/s]

 54%|█████▍    | 225566/414113 [00:53<00:43, 4337.51it/s]

 55%|█████▍    | 226001/414113 [00:54<00:44, 4241.29it/s

 66%|██████▌   | 273246/414113 [01:05<00:33, 4167.95it/s]

 66%|██████▌   | 273666/414113 [01:05<00:33, 4176.72it/s]

 66%|██████▌   | 274084/414113 [01:05<00:33, 4170.75it/s]

 66%|██████▋   | 274502/414113 [01:05<00:33, 4165.24it/s]

 66%|██████▋   | 274919/414113 [01:05<00:33, 4152.92it/s]

 66%|██████▋   | 275335/414113 [01:05<00:33, 4130.02it/s]

 67%|██████▋   | 275749/414113 [01:05<00:33, 4122.16it/s]

 67%|██████▋   | 276162/414113 [01:05<00:33, 4107.69it/s]

 67%|██████▋   | 276573/414113 [01:06<00:33, 4071.28it/s]

 67%|██████▋   | 276986/414113 [01:06<00:33, 4086.51it/s]

 67%|██████▋   | 277395/414113 [01:06<00:33, 4049.78it/s]

 67%|██████▋   | 277804/414113 [01:06<00:33, 4059.97it/s]

 67%|██████▋   | 278223/414113 [01:06<00:33, 4096.14it/s]

 67%|██████▋   | 278633/414113 [01:06<00:33, 4072.33it/s]

 67%|██████▋   | 279041/414113 [01:06<00:33, 4071.81it/s]

 67%|██████▋   | 279477/414113 [01:06<00:32, 4151.67it/s]

 68%|██████▊   | 279905/414113 [01:06<00:32, 4189.27it/s

 79%|███████▉  | 327899/414113 [01:18<00:20, 4235.27it/s]

 79%|███████▉  | 328323/414113 [01:18<00:20, 4225.80it/s]

 79%|███████▉  | 328746/414113 [01:18<00:22, 3776.60it/s]

 79%|███████▉  | 329153/414113 [01:18<00:22, 3859.69it/s]

 80%|███████▉  | 329567/414113 [01:18<00:21, 3937.44it/s]

 80%|███████▉  | 329966/414113 [01:18<00:21, 3940.78it/s]

 80%|███████▉  | 330376/414113 [01:18<00:21, 3985.85it/s]

 80%|███████▉  | 330778/414113 [01:18<00:20, 3976.53it/s]

 80%|███████▉  | 331178/414113 [01:18<00:21, 3947.36it/s]

 80%|████████  | 331576/414113 [01:18<00:20, 3954.61it/s]

 80%|████████  | 331973/414113 [01:19<00:20, 3922.77it/s]

 80%|████████  | 332378/414113 [01:19<00:20, 3957.52it/s]

 80%|████████  | 332797/414113 [01:19<00:20, 4022.27it/s]

 80%|████████  | 333200/414113 [01:19<00:20, 4009.47it/s]

 81%|████████  | 333614/414113 [01:19<00:19, 4047.66it/s]

 81%|████████  | 334053/414113 [01:19<00:19, 4143.17it/s]

 81%|████████  | 334493/414113 [01:19<00:18, 4214.89it/s

 92%|█████████▏| 381522/414113 [01:31<00:07, 4286.08it/s]

 92%|█████████▏| 381975/414113 [01:31<00:07, 4353.83it/s]

 92%|█████████▏| 382416/414113 [01:31<00:07, 4369.32it/s]

 92%|█████████▏| 382854/414113 [01:31<00:07, 4353.45it/s]

 93%|█████████▎| 383290/414113 [01:31<00:07, 4347.13it/s]

 93%|█████████▎| 383732/414113 [01:31<00:06, 4366.53it/s]

 93%|█████████▎| 384169/414113 [01:31<00:06, 4344.38it/s]

 93%|█████████▎| 384604/414113 [01:31<00:06, 4271.17it/s]

 93%|█████████▎| 385032/414113 [01:32<00:06, 4246.44it/s]

 93%|█████████▎| 385457/414113 [01:32<00:06, 4186.78it/s]

 93%|█████████▎| 385877/414113 [01:32<00:06, 4157.91it/s]

 93%|█████████▎| 386294/414113 [01:32<00:06, 4156.37it/s]

 93%|█████████▎| 386715/414113 [01:32<00:06, 4170.04it/s]

 93%|█████████▎| 387167/414113 [01:32<00:06, 4268.01it/s]

 94%|█████████▎| 387616/414113 [01:32<00:06, 4331.25it/s]

 94%|█████████▎| 388067/414113 [01:32<00:05, 4382.29it/s]

 94%|█████████▍| 388520/414113 [01:32<00:05, 4424.30it/s

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [8]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/1618], Loss: 3.9206, Perplexity: 50.42821
Epoch [1/3], Step [200/1618], Loss: 3.7503, Perplexity: 42.5344
Epoch [1/3], Step [300/1618], Loss: 3.3569, Perplexity: 28.7002
Epoch [1/3], Step [400/1618], Loss: 2.8949, Perplexity: 18.08250
Epoch [1/3], Step [500/1618], Loss: 2.8055, Perplexity: 16.5351
Epoch [1/3], Step [600/1618], Loss: 3.0525, Perplexity: 21.1673
Epoch [1/3], Step [700/1618], Loss: 3.0344, Perplexity: 20.7877
Epoch [1/3], Step [800/1618], Loss: 2.4401, Perplexity: 11.4740
Epoch [1/3], Step [900/1618], Loss: 2.6165, Perplexity: 13.6871
Epoch [1/3], Step [1000/1618], Loss: 2.6351, Perplexity: 13.9448
Epoch [1/3], Step [1100/1618], Loss: 2.3958, Perplexity: 10.9766
Epoch [1/3], Step [1200/1618], Loss: 2.4806, Perplexity: 11.9484
Epoch [1/3], Step [1300/1618], Loss: 2.3394, Perplexity: 10.3754
Epoch [1/3], Step [1400/1618], Loss: 2.2358, Perplexity: 9.35418
Epoch [1/3], Step [1500/1618], Loss: 2.3062, Perplexity: 10.0365
Epoch [1/3], Step [1600/1618], L

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.